In [1]:
import os
import sys
from pathlib import Path

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import torch
from PIL import Image
from tqdm import trange

sys.path.append("..")

from core.config import default_config
from core.extract_video_rgb import extract_video_rgb
from core.image_segmentor import load_sam_auto_gen, segment_with_sam
from core.mask_handler import MaskHandler
from core.propagator import load_sam2_video_predictor_and_initial_state
from core.task_handler import TaskHandler
from core.utils import ID2RGBConverter, flatten_mask, viz_mask

### Step 1: extract rgb into a folder

In [2]:
video_path = "/scratch/quanta/Experiments/Thesis/exps/prob_video_4_open_laptop/viz/rgb.mp4"
work_dir = Path(
    "/scratch/quanta/Experiments/Thesis/exps/sam2_full_video_track_example")

In [3]:
extract_video_rgb(
    video_path=video_path,
    save_workspace_dir=work_dir,
)

100%|██████████| 708/708 [00:08<00:00, 85.07it/s]


### Step 2: Key frame image segmentation

In [4]:
sam = load_sam_auto_gen(
    ckpt_pth="/scratch/quanta/Models/SAM/sam_vit_h_4b8939.pth",
    points_per_side=default_config['sam1_points_per_side'],
    pred_iou_thresh=default_config['sam1_pred_iou_thrshold'],
)

In [5]:
segment_with_sam(
    rgb_dir=str(work_dir / "data/rgb"),
    save_dir=str(work_dir / "data/sam_1_seg"),
    sam_auto_gen=sam,
    min_size=default_config['min_size'],
    step=default_config['step'],
    max_masks_per_frame=default_config['max_masks_per_frame'],
)

100%|██████████| 142/142 [20:37<00:00,  8.71s/it]


In [6]:
del sam
torch.cuda.empty_cache()

### Step 3: Use SAM2 to propagate and associate all masks

In [7]:
config_pth = "configs/sam2.1/sam2.1_hiera_l.yaml"
ckpt_pth = "/scratch/quanta/Models/SAM2/sam2.1_hiera_large.pt"
device = "cuda:0"
video_predictor, inference_state = load_sam2_video_predictor_and_initial_state(
    ckpt_pth=ckpt_pth,
    rgb_jpg_dir=str(work_dir / "data/rgb"),
    model_config=config_pth,
    device=device,
)

frame loading (JPEG):   1%|          | 5/708 [00:00<00:17, 39.78it/s]

In [8]:
task_handler = TaskHandler(
    queue_dir=str(work_dir / "data/sam_2_queue"),
    sam_mask_dir=str(work_dir / "data/sam_1_seg"),
    save_mask_dir=str(work_dir / "data/sam_2_track"),
    video_predictor=video_predictor,
    inference_state=inference_state,
    step=default_config['step'],
    disappear_thresh=default_config['disappear_threshold'],
    iou_thresh=default_config['iou_threshold'],
    device=device,
)

In [9]:
task_handler.submit_initial_tasks()

142it [00:33,  4.21it/s]
142it [00:17,  8.02it/s]


frame loading (JPEG): 100%|██████████| 708/708 [00:24<00:00, 29.13it/s]


In [ ]:
task_return = True
while task_return is True:
    torch.cuda.empty_cache()
    task_return = task_handler.run_one_task()

In [11]:
# load id_map and union find
id_map = {}

import json
with open(str(work_dir / "data/sam_2_queue/id_map.json")) as f:
    tmp_map = json.load(f)

for i in tmp_map.keys():
    j = i
    while tmp_map[str(j)] != j:
        j = tmp_map[str(j)]

    id_map[int(i)] = j

with open(str(work_dir / "data/sam_2_queue/united_id_map.json"), "w") as f:
    json.dump(id_map, f, indent=4)

### Step 4: Visualize

In [12]:
converter = ID2RGBConverter()
mask_handler = MaskHandler(str(work_dir / "data/sam_2_track"))

viz_save_pth = work_dir / "temp/viz_sam2_association"
viz_save_pth.mkdir(parents=True, exist_ok=True)

In [13]:
for i in trange(task_handler.num_frames):
    masks_data = mask_handler.load_masks(i)
    obj_ids = [id_map[item['original_obj_id']] for item in masks_data]
    viz_img = viz_mask(
        flattened_mask=flatten_mask(
            mask=masks_data,
            object_id_list=obj_ids,
        ),
        converter=converter,
    )
    Image.fromarray(viz_img).save(str(viz_save_pth / "{:06d}.png".format(i)))

100%|██████████| 708/708 [15:49<00:00,  1.34s/it]


In [15]:
import ffmpeg

(work_dir / "viz").mkdir(exist_ok=True, parents=True)

(ffmpeg.input(
    str(viz_save_pth / "*.png"),
    pattern_type="glob",
    framerate=30,
).output(
    str(work_dir / "viz/sam2_assotiation.mp4")
).run())

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.3.0 (conda-forge gcc 13.3.0-1)
  configuration: --prefix=/home/quanta/.conda/envs/sam2 --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1727723455708/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1727723455708/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1727723455708/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1727723455708/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --disable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enable-vaapi --enable-libopenvino --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav

(None, None)